### Script for DDPHE

Data pre-processing:

* Obtain hourly data from the Canary-S and AirNow sensors, via http://airsense.sensibleiot.com/
* Remove all missing/negative values from the data
* Extract the hour (1-24) and month (1-12) from the DateTime variable
* Perform the following on the Hour and Month variables, respectively: cos(2*pi*Hour/24), cos(*pi*Month/12)
* Determine whether the observation occurred on a weekend, based on the date --> boolean indicator
* Extract the length of arterial roads in a 500-meter buffer of each monitor location (there is R code for this, if and when the list needs to be updated)

Note: Road files can be found at https://drive.google.com/drive/folders/1kNEF3-kntNel9LpZvMZnKLnHJ3Fg0_gO?usp=sharing

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.externals import joblib
from sklearn import linear_model
import pandas as pd

C:\Users\ellen\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
## Long-term calibration (random forest):

#Read in data. I will call the data frame "DATA".
X = DATA #data from collocated monitors; includes Canary-S PM2.5, temp, RH, cos(*pi*month/12), cos(2*pi*hour/24), 
                                        # weekend indicator, arterial roads in 500m buffer
y = AirNow

#Train
pipe = make_pipeline(RandomForestRegressor(n_estimators = 250, min_samples_leaf = 5, max_features = 4, 
                                           bootstrap = True, random_state=0))
pipe.fit(X, y)
joblib.dump(pipe, 'Calibration_model.pkl')

#Loading the saved model with joblib
pipe = joblib.load('Calibration_model.pkl')

# New data to predict
pr = pd.read_csv('Archived_data.csv')

# apply the whole pipeline to data
predictions = pd.Series(pipe.predict(pr))

In [ ]:
## Short-term regression (multiple linear regression):

x = DATA #data from last three weeks; includes Canary-S PM2.5, temp, RH, and arterial roads in 500m buffer
y = AirNow

#Fit model
lm = linear_model.LinearRegression()
model = lm.fit(x,y)

#Predict
new_data #data from this week; includes Canary-S PM2.5, temp, RH, and arterial roads in 500m buffer
predictions = lm.predict(new_data)